# Linear Solver Performance: Constant Coefficient Poisson, MPI-parallel

### Part 1, Benchmark Setup and Execution

In [ ]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

Execution Date/time is 13.07.2022 07:35:43


In [ ]:
#r "BoSSSpad.dll"
//#r "../../../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
using BoSSS.Application.SipPoisson;

In [ ]:
wmg.Init("WIP-k-LinslvPerfPar");
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

Project name is set to 'WIP-k-LinslvPerfPar'.
Creating database 'X:\WIP-k-LinslvPerfPar'.


In [ ]:
ExecutionQueues

## Utility definitions

In [ ]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
}

## Init grids and save to database

In [ ]:
wmg.Grids

Creating database 'C:\BoSSS-Databases\WIP-k-LinslvPerfPar'.
Creating database 'X:\jenkins\databases\WIP-k-LinslvPerfPar'.


Create meshes in various resolutions:
- domain $\Omega = (0,10) \times (-1,1) \times (-1,1)$; 
- a Dirichlet boundary is set for $x = 0$, everywhere else Neumann boundaries are assumed; 
  these are typically more challenging for a linear solver than Dirichlet boundaries.
- in order to make the problem a bit challenging, a non-equidistant spacing for $y$ and $z$ coordinate are used
- the domain shape is reminiscent of a typical grid for the simulation of channel flows, with exception of the boundary conditions

In [ ]:
int[] Resolutions_3D = new int[] { 4, 8, 16, 24, 32, 48, 64, 128 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];

for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(0, 10, Res*5 + 1);
    double[] yNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    double[] zNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = wmg.CurrentProject + "-SIP_Poisson_J" + J;
    
    //grids[cnt] = null;
    //foreach(IGridInfo grd in wmg.Grids) { 
    //    bool check = grd.Name.Contains(string.Format("_J"+ J));
    //    if(check) {
    //      grids[cnt] = grd;
    //    }
    //}
    
    Console.WriteLine("Creating Grid: " + GridName);
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(GridName)); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null) { // 
        Console.WriteLine($"Creating grid with {J} = {Res*5}x{Res}x{Res} cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
    
        g.DefineEdgeTags(delegate (double[] X) {
            if(Math.Abs(X[0] - 0.0) <= 1.0e-6)
                return BoundaryType.Dirichlet.ToString();
            else
                return BoundaryType.Neumann.ToString();
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

Creating Grid: WIP-k-LinslvPerfPar-SIP_Poisson_J320
Creating grid with 320 = 20x4x4 cells.
Grid Edge Tags changed.
Creating Grid: WIP-k-LinslvPerfPar-SIP_Poisson_J2560
Creating grid with 2560 = 40x8x8 cells.
Grid Edge Tags changed.
Creating Grid: WIP-k-LinslvPerfPar-SIP_Poisson_J20480
Creating grid with 20480 = 80x16x16 cells.
Grid Edge Tags changed.
Creating Grid: WIP-k-LinslvPerfPar-SIP_Poisson_J69120
Creating grid with 69120 = 120x24x24 cells.
Grid Edge Tags changed.
Creating Grid: WIP-k-LinslvPerfPar-SIP_Poisson_J163840
Creating grid with 163840 = 160x32x32 cells.
Grid Edge Tags changed.
Creating Grid: WIP-k-LinslvPerfPar-SIP_Poisson_J552960
Creating grid with 552960 = 240x48x48 cells.
Grid Edge Tags changed.
Creating Grid: WIP-k-LinslvPerfPar-SIP_Poisson_J1310720
Creating grid with 1310720 = 320x64x64 cells.
Grid Edge Tags changed.
Creating Grid: WIP-k-LinslvPerfPar-SIP_Poisson_J10485760
Creating grid with 10485760 = 640x128x128 cells.
Grid Edge Tags changed.


In [ ]:
grids

#0: { Guid = ca6697d0-bdf5-4ebb-82ff-246fefee78d4; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J320; Cell Count = 320; Dim = 3 }
#1: { Guid = e10232b0-f45a-44df-974d-0c1f3dba88c8; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J2560; Cell Count = 2560; Dim = 3 }
#2: { Guid = 913bc8c5-7e88-4b14-b63f-e2279ecb253f; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J20480; Cell Count = 20480; Dim = 3 }
#3: { Guid = 8b71978f-5cae-4cba-9800-949eebe01874; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J69120; Cell Count = 69120; Dim = 3 }
#4: { Guid = 604a6c58-711d-46b9-bf21-c65e9d5598c4; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J163840; Cell Count = 163840; Dim = 3 }
#5: { Guid = 69930cf6-1f36-4bde-a0a8-d3a0a95ff338; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J552960; Cell Count = 552960; Dim = 3 }
#6: { Guid = c41a4871-4a08-4615-bd7b-025b5bcfb02f; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J1310720; Cell Count = 1310720; Dim = 3 }
#7: { Guid = afc87846-7cf4-42d1-afac-2023e8c90920; Name = WIP-k-LinslvPerfPar-SIP_Poisson_J10485760; C

## Setup Control Object for a Solver Run

### Values/Formulas for the Values of RHS and Boundary Conditions

In [ ]:
string nl = System.Environment.NewLine;
string math_expressions_code = 
"static class InitialValues { " +
"    public static double RHS(double[] X) { " + nl +
"        return -Math.Sin(X[0]); " + nl +
"    } " + nl +
"     " + nl +
"    public static double DirichletBC(double[] X) { " + nl +
"        return 0.0; " + nl +
"    } " + nl +
"     " + nl +
"    public static double NeumannBC(double[] X) { " + nl +
"       if(Math.Abs(X[1] - 1.0) < 1.0e-8 || Math.Abs(X[1] + 1.0) < 1.0e-8) " + nl +
"           return 0; " + nl +
"       if(X.Length > 2 && (Math.Abs(X[2] - 1.0) < 1.0e-8 || Math.Abs(X[2] + 1.0) < 1.0e-8)) " + nl +
"           return 0; " + nl +
"     " + nl +
"       return Math.Cos(10.0); " + nl +
"   } " + nl +
"} " + nl;

In [ ]:
var RHS_value = new Formula("InitialValues.RHS", false, math_expressions_code);
var NeumannBC_value = new Formula("InitialValues.NeumannBC", false, math_expressions_code);
var DirichletBC_value = new ConstantValue(0.0);

Brief test of the boundary values:

In [ ]:
RHS_value.Evaluate(new Vector(0.2, 0.1), 0)

-0.19866933079506122

In [ ]:
NeumannBC_value.Evaluate(new Vector(0.2, 0.1), 0)

-0.8390715290764524

### Setup of Parameter Study

Polynomial degrees to test:

In [ ]:
//int[] PolyDegS = new int[] {2, 3, 4, 5, 6};
int[] PolyDegS = new int[] {2, 3, 5}; // reduced options for dev.

Solvers which we want to instrument:

In [ ]:
using BoSSS.Solution.AdvancedSolvers;

In [ ]:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.exp_Kcycle_schwarz
}; 

Number of processors:

In [ ]:
int[] MPIsizes = new int[] { 1, 2, 4, 8, 16, 32, 64, 128 };

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [ ]:
//int GetMaxAllowedDOF(LinearSolverCode code) {
//    switch(code) {
//        case LinearSolverCode.direct_pardiso:
//        case LinearSolverCode.direct_mumps:
//        return 1100000; // 1.1 Million for iterative solvers at maximum
//    
//        default: 
//        return 30000000; // Up to 30 Million for iterative solvers
//    }
//}

Loop over all combinations of parameters and define a control object for each combo:

In [ ]:
var controls = new List<(SipControl ctrl, int NoOfProcs)>();
LinearSolverCode solver_name = LinearSolverCode.exp_Kcycle_schwarz;
foreach(int k in PolyDegS) {   
foreach(IGridInfo grd in grids) {
foreach(int MPIsize in MPIsizes) {
    int Np = Utils.Np(k);
    int J  = grd.NumberOfCells;
    if(J / MPIsize < 16) {
        // less than 16 cells per processor - to low for a multigrid.
        continue; 
    }
    if(J*Np/MPIsize > 500000) {
        // not interested in doing more then 500'000 DOFs per processor
        continue;
    } 
    var ctrl = new SipControl();
    controls.Add((ctrl, MPIsize));
       
    
    string caseName = string.Format("Poisson_J{0}_k{1}_Sz{2}", J, k, MPIsize);
    Console.WriteLine("setting up: " + caseName);
    ctrl.SessionName = "SIP_" + caseName;

    ctrl.SetGrid(grd);
    ctrl.savetodb = true;
    ctrl.SetDGdegree(k);

    ctrl.LinearSolver           = solver_name.GetConfig();
    var isc = ctrl.LinearSolver as IterativeSolverConfig;
    if(isc != null) {
        //Console.WriteLine(isc.ConvergenceCriterion);
        //ctrl.LinearSolver.TargetBlockSize      = Math.Min(J*Np-1,10000);
        isc.ConvergenceCriterion = 1e-8;
    }
        
        
    ctrl.InitialValues.Add("RHS", RHS_value);
    ctrl.AddBoundaryValue(BoundaryType.Dirichlet.ToString(), "T", DirichletBC_value);
    ctrl.AddBoundaryValue(BoundaryType.Neumann.ToString(), "T", NeumannBC_value);
    
    ctrl.TracingNamespaces = "*";
}
}
}

setting up: Poisson_J320_k2_Sz1
setting up: Poisson_J320_k2_Sz2
setting up: Poisson_J320_k2_Sz4
setting up: Poisson_J320_k2_Sz8
setting up: Poisson_J320_k2_Sz16
setting up: Poisson_J2560_k2_Sz1
setting up: Poisson_J2560_k2_Sz2
setting up: Poisson_J2560_k2_Sz4
setting up: Poisson_J2560_k2_Sz8
setting up: Poisson_J2560_k2_Sz16
setting up: Poisson_J2560_k2_Sz32
setting up: Poisson_J2560_k2_Sz64
setting up: Poisson_J2560_k2_Sz128
setting up: Poisson_J20480_k2_Sz1
setting up: Poisson_J20480_k2_Sz2
setting up: Poisson_J20480_k2_Sz4
setting up: Poisson_J20480_k2_Sz8
setting up: Poisson_J20480_k2_Sz16
setting up: Poisson_J20480_k2_Sz32
setting up: Poisson_J20480_k2_Sz64
setting up: Poisson_J20480_k2_Sz128
setting up: Poisson_J69120_k2_Sz2
setting up: Poisson_J69120_k2_Sz4
setting up: Poisson_J69120_k2_Sz8
setting up: Poisson_J69120_k2_Sz16
setting up: Poisson_J69120_k2_Sz32
setting up: Poisson_J69120_k2_Sz64
setting up: Poisson_J69120_k2_Sz128
setting up: Poisson_J163840_k2_Sz4
setting up: Poi

In [ ]:
//string path = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\ipPoisson\bin\Release\net5.0\PoissonBench";
//foreach(var ctrl in controls) {
//    ctrl.savetodb = false;
//    ctrl.SaveToFile(System.IO.Path.Combine(path, ctrl.SessionName + ".obj"));
//}

In [ ]:
controls.Count()

107

## Launch Jobs

Use the default queue defined on this machine:

In [ ]:
ExecutionQueues

In [ ]:
var myBatch = GetDefaultQueue();
myBatch

Username,Password,ServerName,PrivateKeyFilePath,AdditionalBatchCommands,ExecutionTime,DeploymentBaseDirectoryAtRemote,RuntimeLocation,SlurmAccount,Email,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,MonoDebug,AllowedDatabasesPaths
fk69umer,<null>,lcluster16.hrz.tu-darmstadt.de,C:\Users\flori\.ssh\id_rsa,"[ #SBATCH -C avx512, #SBATCH --mem-per-cpu=8000 ]",05:00:00,/work/scratch/fk69umer/bosss_deploy,linux/amd64-openmpi,special00006,<null>,X:\bosss_deploy,False,Lb2-specialPrj,dotnet,False,[ X: == /work/scratch/fk69umer ]


In [ ]:
foreach((var ctrl, int MPIsize) in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName);
    var j = ctrl.CreateJob();
    j.RetryCount = 1;
    j.NumberOfMPIProcs = MPIsize;
    j.Activate(myBatch);
    //ctrl.RunBatch(); // run in the default queue for this project
}

 Submitting: SIP_Poisson_J320_k2_Sz1
Deploying job SIP_Poisson_J320_k2_Sz1 ... 
Deploying executables and additional files ...
Deployment directory: X:\bosss_deploy\WIP-k-LinslvPerfPar-ipPoisson2022Jul13_101252
copied 43 files.
   written file: control.obj
deployment finished.
stderr received: Lmod: loading emacs 27.1 
Lmod: loading git 2.35.1 
Lmod: loading gcc 8.5.0 
Lmod: loading cmake 3.19.3 
Lmod: loading mono 6.12.0 
Lmod: loading openucx 1.12.0 
Lmod: loading openmpi 4.0.5 
Lmod: loading python 3.9.5 
----- (end of stderr)
stderr received: Lmod: loading emacs 27.1 
Lmod: loading git 2.35.1 
Lmod: loading gcc 8.5.0 
Lmod: loading cmake 3.19.3 
Lmod: loading mono 6.12.0 
Lmod: loading openucx 1.12.0 
Lmod: loading openmpi 4.0.5 
Lmod: loading python 3.9.5 
----- (end of stderr)

 Submitting: SIP_Poisson_J320_k2_Sz2
Deploying job SIP_Poisson_J320_k2_Sz2 ... 
Deploying executables and additional files ...
Deployment directory: X:\bosss_deploy\WIP-k-LinslvPerfPar-ipPoisson2022Jul13_1

In [ ]:
//foreach(var j in wmg.AllJobs.Values)
//    j.DeleteOldDeploymentsAndSessions();

### Wait for Completion and Check Job Status

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

In [ ]:
wmg.AllJobs

#0: SIP_Poisson_J320_k2_Sz1: FinishedSuccessful (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)	SIP_Poisson_J320_k2_Sz1: FinishedSuccessful (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)
#1: SIP_Poisson_J320_k2_Sz2: FinishedSuccessful (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)	SIP_Poisson_J320_k2_Sz2: FinishedSuccessful (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)
#2: SIP_Poisson_J320_k2_Sz4: FinishedSuccessful (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)	SIP_Poisson_J320_k2_Sz4: FinishedSuccessful (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)
#3: SIP_Poisson_J320_k2_Sz8: FinishedSuccessful (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm

In [ ]:
int succJob = wmg.AllJobs.Values.Where(job => job.Status == JobStatus.FinishedSuccessful).Count();
int succSess = wmg.Sessions.Where(Si => Si.Name.Contains("SIP_Poisson") && (Si.SuccessfulTermination == true)).Count();
(succJob, succSess)

Item1,Item2
86,86


In [ ]:
int failJob = wmg.AllJobs.Values.Where(job => job.Status != JobStatus.FinishedSuccessful).Count();
int failSess = wmg.Sessions.Where(Si => Si.Name.Contains("SIP_Poisson") && (Si.SuccessfulTermination == false)).Count();
(failJob, failSess)

Item1,Item2
21,21


In [ ]:
NUnit.Framework.Assert.AreEqual(succJob + failJob, controls.Count);
NUnit.Framework.Assert.AreEqual(succJob, succSess);
NUnit.Framework.Assert.AreEqual(failJob, failSess);

In [ ]:
wmg.Sessions.Where(sess => sess.Name.StartsWith("SIP_Poisson_J"))

#0: WIP-k-LinslvPerfPar	SIP_Poisson_J552960_k5_Sz128*	07/13/2022 10:49:28	92800555...
#1: WIP-k-LinslvPerfPar	SIP_Poisson_J552960_k5_Sz64*	07/13/2022 10:49:12	dbbc50bc...
#2: WIP-k-LinslvPerfPar	SIP_Poisson_J163840_k5_Sz128*	07/13/2022 10:48:55	3f014201...
#3: WIP-k-LinslvPerfPar	SIP_Poisson_J69120_k5_Sz128	07/13/2022 10:46:48	7c432a19...
#4: WIP-k-LinslvPerfPar	SIP_Poisson_J69120_k5_Sz64	07/13/2022 10:46:06	a4c72fa1...
#5: WIP-k-LinslvPerfPar	SIP_Poisson_J163840_k5_Sz64*	07/13/2022 10:46:51	ae712711...
#6: WIP-k-LinslvPerfPar	SIP_Poisson_J20480_k5_Sz128	07/13/2022 10:42:58	a22bd47c...
#7: WIP-k-LinslvPerfPar	SIP_Poisson_J163840_k5_Sz32*	07/13/2022 10:44:06	becb5265...
#8: WIP-k-LinslvPerfPar	SIP_Poisson_J69120_k5_Sz32*	07/13/2022 10:43:36	19da36da...
#9: WIP-k-LinslvPerfPar	SIP_Poisson_J69120_k5_Sz16*	07/13/2022 10:42:58	ae7252de...
#10: WIP-k-LinslvPerfPar	SIP_Poisson_J20480_k5_Sz64	07/13/2022 10:40:05	72f46beb...
#11: WIP-k-LinslvPerfPar	SIP_Poisson_J69120_k5_Sz8*	07/13/2022 10:41:0

In [ ]:
//controls.Select(tuple => tuple.ctrl)
//        .Select(ctrl => (ctrl,ctrl.GetJob()))
//        .Select(ttt => (ttt.Item1.SessionName == ttt.Item2.Name, ttt.Item1.SessionName, ttt.Item2.Name))

In [ ]:
var NoSuccess = wmg.AllJobs.Values
                        .Where(job => job.Status != JobStatus.FinishedSuccessful)
                        .ToArray();
NoSuccess

#0: SIP_Poisson_J163840_k2_Sz4: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)
#1: SIP_Poisson_J552960_k2_Sz16: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)
#2: SIP_Poisson_J1310720_k2_Sz32: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)
#3: SIP_Poisson_J20480_k3_Sz1: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)
#4: SIP_Poisson_J69120_k3_Sz4: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)
#5: SIP_Poisson_J163840_k3_Sz8: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)
#6: SIP_Poisson_J163840_k3_Sz16: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.t

In [ ]:
wmg.AllJobs.Values
        .Where(job => job.Name.Contains("SIP_Poisson_J552960_k2"))
        .Select(job => (job.Name, job.Status, job.LatestDeployment?.DeploymentDirectory?.FullName))
        .ToArray()

index,Item1,Item2,Item3
0,SIP_Poisson_J552960_k2_Sz16,FailedOrCanceled,X:\bosss_deploy\WIP-k-LinslvPerfPar-ipPoisson2022Jul13_102111
1,SIP_Poisson_J552960_k2_Sz32,FinishedSuccessful,X:\bosss_deploy\WIP-k-LinslvPerfPar-ipPoisson2022Jul13_102126
2,SIP_Poisson_J552960_k2_Sz64,FinishedSuccessful,X:\bosss_deploy\WIP-k-LinslvPerfPar-ipPoisson2022Jul13_102142
3,SIP_Poisson_J552960_k2_Sz128,FinishedSuccessful,X:\bosss_deploy\WIP-k-LinslvPerfPar-ipPoisson2022Jul13_102158


In [ ]:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail + ":  @" + ((fail.LatestDeployment?.DeploymentDirectory?.FullName) ?? " no deployment directory"));

SIP_Poisson_J163840_k2_Sz4: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006):  @X:\bosss_deploy\WIP-k-LinslvPerfPar-ipPoisson2022Jul13_101938
SIP_Poisson_J163840_k2_Sz4: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006):  @X:\bosss_deploy\WIP-k-LinslvPerfPar-ipPoisson2022Jul13_101938
SIP_Poisson_J163840_k2_Sz4: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006):  @X:\bosss_deploy\WIP-k-LinslvPerfPar-ipPoisson2022Jul13_101938
SIP_Poisson_J163840_k2_Sz4: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006):  @X:\bosss_deploy\WIP-k-LinslvPerfPar-ipPoisson2022Jul13_101938
SIP_Poisson_J163840_k2_Sz4: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006):  @X:\bosss_dep

In [ ]:
/*
string PathOffset = @"C:\Users\jenkinsci\Desktop\LinSlvPerfFail-20apr22";
foreach(var fail in NoSuccess) {
    var C = fail.GetControl();
    C.savetodb = false;
    C.SaveToFile(System.IO.Path.Combine(PathOffset, fail.Name + ".obj"));
    
    string Stdout = fail.Stdout;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stdout.txt"), Stdout);
    
    string Stderr = fail.Stderr;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stderr.txt"), Stderr);
}
*/

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => Si.Name.Contains("SIP_Poisson") &&
                                        (Si.SuccessfulTermination == false
                                        || Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0));
FailedSessions

In [ ]:
//foreach(var s in FailedSessions)
//    s.Delete(true);

In [ ]:
//wmg.AllJobs["SIP_Poisson_J2560_k5_pMultigrid"].AllDeployments

In [ ]:
NUnit.Framework.Assert.Zero(NoSuccess.Count(), "Some Jobs Failed");

In [ ]:
//foreach(Job jF in NoSuccess)
//   jF.DeleteOldDeploymentsAndSessions();

In [ ]:
NUnit.Framework.Assert.Zero(FailedSessions.Count(), "Some Sessions did not derminate successfully.");

List Output of some job (arbitrarily the first one):

In [ ]:
wmg.AllJobs.First().Value.Stdout

## Create Table for Post-Processing

(moved to post-processing worksheet)

In [ ]:
/*
// evaluators to add additional columns to the session table
static class AddCols {
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("MatrixAssembly");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Aggregation_basis_init");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Init");
        var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Run");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
        }
}
*/

In [ ]:
/*
wmg.AdditionalSessionTableColums.Clear();
wmg.AdditionalSessionTableColums.Add("MatrixAssembly", AddCols.XdgMatrixAssembly_time);
wmg.AdditionalSessionTableColums.Add("Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Init_time", AddCols.Solver_Init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Run_time", AddCols.Solver_Run_time);
wmg.AdditionalSessionTableColums.Add("NoOfCores", AddCols.NoOfCores);
*/

In [ ]:
/*
var FullSessTab = wmg.SessionTable;

// The Session column can't be serialized,
// we have to remove it
List<string> AllCols = FullSessTab.GetColumnNames().ToList();
AllCols.Remove("Session");
FullSessTab = FullSessTab.ExtractColumns(AllCols.ToArray());
*/

In [ ]:
//foreach(var s in AllCols) Console.WriteLine(s);

In [ ]:
//foreach(var s in wmg.Sessions[0].KeysAndQueries.Keys) Console.WriteLine(s + " \t = " + wmg.Sessions[0].KeysAndQueries[s]);

In [ ]:
/*var SubTab = FullSessTab.ExtractColumns(
    "SessionName","DGdegree:T", "Grid:NoOfCells", "LinearSolver.SolverCode", "DOFs", "MatrixAssembly",
    "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time", "NoIter");
*/

In [ ]:
//SubTab.Print();

In [ ]:
/*
// Filename
var now               = DateTime.Now;
FullSessTab.TableName = wmg.CurrentProject + "_" + now.Year + "-" + now.Month + "-" + now.Day;
string docpath        = FullSessTab.TableName + ".json";
SubTab.SaveToFile(wmg.CurrentProject + ".json");
SubTab.ToCSVFile(wmg.CurrentProject + ".csv");
*/